<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/LSHADE/testLSHADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats
import random

In [2]:
# HappyCat function
# global mimima 0 at (-1,-1,...,-1)
# Not convex and non separable it is usually evaluated at xi =[(-600,600),..(-600,600)]

def fobj(x): 
    alpha=1. / 8
    s = sum(x**2) 
    return ((s - len(x))**2)**alpha + (s / 2 + sum(x)) / len(x) + 0.5 

In [3]:
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]

In [4]:
def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H = setTUNE
  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,p, terminal, Narquive,H] 
  dim = X.shape[1]
  best_number = int(p*popsize)

  justGoOUT=1

  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break

      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      iused = np.random.choice(idxs, 1, replace = False)
      idxs = [idx for idx in range(popsize) if (idx != iused and idx != best_idx)]
      a  = X[iused]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X[idxs,:],Xii,axis=0)
      else:
        Xnew=np.copy(X[idxs,:])

      idxs2 = [idx for idx in range(len(Xnew[:,0])) if 2 > 1]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (a - b)
      mutant = mutant.ravel()

      trial = np.copy(mutant)
    


      for j in range(dim):
        rdn = np.random.rand()
        if(rdn < crossp) : 
          trial[j] = mutant[j] # not necessary but to make clear (trial = np.copy(mutant))
        else:
          trial[j] = X[i,j]

      mutant  = np.copy(trial)
      
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]

      Xmutant.append(mutant)

      FES = FES + 1
      fmut =fobj(mutant)
      fmutant.append(fmut)


    for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fitness[i]=fmutant[i]
        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it
      if( fmutant[i]  < fitness[i]):
        if(len(Xarq[:,0])< Narquive):
          Xarq=np.append(Xarq,Xold,axis=0)
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary line 19
          Xarq[ri,:] = Xold

        SCR.append(crossp)
        SF.append(mut)
        fx.append(fitness[i])
        fu.append(fmutant[i])
    # memory update algorithm 1:
    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      w = deltaF/sum(deltaF)
      if(MCR[kH] == terminal or max(SRC)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1
      if(kH > H): kH = 0
    else:
      MCR = MCR
      MF = MF
  
  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
  

  y=fitness


  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:dim]
  XY=XYsorted
  fitness = XYsorted[:,-1]  
  BEST=X[0,:]
  FOBEST=fitness[0]
  BEST_XY =np.append(BEST,FOBEST)
  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [5]:
cols=30
dim = cols
NPAR1 = 50
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)
popsize = NPAR1
X=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

FES = 0
X1 = np.copy(X)
fit1 = np.asarray([fobj(ind) for ind in X1])
FES = FES + NPAR1
best_idx = np.argmin(fit1)
best = X1[best_idx]
fbest = fit1[best_idx] 
BEST1,FOBEST1 = best, fbest

In [6]:
for i in range(H):
  MCR.append(0.5);MF.append(0.5) # line 3
Narquive = popsize
setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]

In [7]:
for i in range(100):
  #print(i,X1.shape)
  fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
  print('i=',i,FOBEST1)

i= 0 1084.657952705128
i= 1 799.2506802534373
i= 2 799.2506802534373
i= 3 760.2024328993866
i= 4 642.551537389101
i= 5 619.5424435034377
i= 6 334.454122985829
i= 7 317.4153529805391
i= 8 317.4153529805391
i= 9 317.4153529805391
i= 10 221.12894484807077
i= 11 221.12894484807077
i= 12 208.90411912270545
i= 13 208.90411912270545
i= 14 208.90411912270545
i= 15 208.90411912270545
i= 16 206.61698606971672
i= 17 205.7532297838732
i= 18 164.29404953262636
i= 19 151.7261973058015
i= 20 151.7261973058015
i= 21 106.26634520101143
i= 22 106.26634520101143
i= 23 106.26634520101143
i= 24 95.59665557836894
i= 25 78.63915423562042
i= 26 78.63915423562042
i= 27 78.63915423562042
i= 28 58.36381580078507
i= 29 58.36381580078507
i= 30 58.36381580078507
i= 31 53.703572775370894
i= 32 51.47868177327221
i= 33 51.47868177327221
i= 34 50.33593731562404
i= 35 39.2406826976962
i= 36 39.2406826976962
i= 37 39.2406826976962
i= 38 38.16122234878154
i= 39 34.06780810716023
i= 40 33.22584571037774
i= 41 23.7187877064

In [8]:
X1.shape

(50, 30)

In [12]:
fobj(X1[0,:])

0.7810492090881442

In [10]:
fit1

array([0.78104921, 0.84163994, 0.91210738, 0.93371206, 0.95080437,
       0.95945615, 0.99370681, 0.99838668, 1.02327714, 1.02833676,
       1.03958755, 1.06239737, 1.07799935, 1.09131897, 1.09624405,
       1.09657517, 1.09812768, 1.13118958, 1.13176751, 1.17468814,
       1.18630319, 1.19616756, 1.21904012, 1.22576331, 1.24280479,
       1.25844649, 1.27142468, 1.28407091, 1.31574663, 1.31703816,
       1.32590524, 1.33406398, 1.33502973, 1.33598674, 1.33881754,
       1.36269506, 1.37638766, 1.37703567, 1.37865635, 1.40512781,
       1.40822847, 1.41019587, 1.43683039, 1.52183273, 1.52375894,
       1.53824826, 1.5419801 , 1.54542075, 1.73216368, 1.97512384])